In [8]:
import os
import pandas as pd
import numpy as np
import traceback

In [9]:
from langchain.chat_models import ChatOpenAI
from dotenv import load_dotenv
load_dotenv()

True

In [10]:
llm=ChatOpenAI(openai_api_key=os.getenv("OPENAI_API_KEY"),model_name="gpt-3.5-turbo", temperature=0.5)